In [5]:
!pip install datasets trl transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os
from huggingface_hub import login

token = os.getenv("token")

login(
  token=token, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [1]:
# import os 
# os.environ["WANDB_DISABLED"] = "true" # colab

from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM

model_name = "t5-small"  # "t5-base", "t5-large", "t5-3b", "t5-11b" # cuda out of memory in puffer with using 4 gpus
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset", split="train[:10%]") # 1k

training_args = DPOConfig(output_dir="t5_small_DPO_ds302", logging_steps=10)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Extracting prompt in train dataset:   0%|          | 0/1088 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1088 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1088 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,8.046100
20,5.078600
30,4.482300
40,6.678900
50,9.901000
60,5.190300
70,3.795300
80,3.101600
90,6.350900
100,4.188600


TrainOutput(global_step=408, training_loss=5.291200193704343, metrics={'train_runtime': 279.2023, 'train_samples_per_second': 11.69, 'train_steps_per_second': 1.461, 'total_flos': 0.0, 'train_loss': 5.291200193704343, 'epoch': 3.0})

In [4]:
model.push_to_hub("EraCoding/DPO_a5_nlp")

# model = AutoModelForSeq2SeqLM.from_pretrained("nielsr/my-awesome-bert-model")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/EraCoding/DPO_a5_nlp/commit/ef08b3c9ec0c49d4ebe535b7b63bd743cf3c4120', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='ef08b3c9ec0c49d4ebe535b7b63bd743cf3c4120', pr_url=None, repo_url=RepoUrl('https://huggingface.co/EraCoding/DPO_a5_nlp', endpoint='https://huggingface.co', repo_type='model', repo_id='EraCoding/DPO_a5_nlp'), pr_revision=None, pr_num=None)

In [ ]:
import tensorboard